In [1]:
import random
import numpy as np
import time

from Library import ACS_Ant, Solve


class CPPInstance:
    def __init__(self, n,distance,goal_value,graph):
        self.n = n
        self.goal_value = goal_value
        self.distance = distance

        self.starting_point = 0
        self.graph = graph

    def getNumVertices(self):
        return self.n

    def getPossbileNextEdgesByVertex(self, vertex):
        return self.graph[vertex][self.graph[vertex] != 0]

    def getStartPoint(self):
        return 0

    def getGraph(self):
        return self.graph

    def getNextVertex(self, current_vertex, next_edge):
        return self.graph[current_vertex][next_edge]

    def get_goal_value(self):
        return self.goal_value

    def get_distance(self):
        return self.distance


class CPPAnt(ACS_Ant):
    def __init__(self, instance, **kwargs):
        self.instance = instance
        self.value = 0
        self.last_node = 0
        #self.coin_amount = 0
        self.chosen_coins = [0]*len(self.instance.getGraph()[0]) #wie len(d)

        super().__init__(**kwargs)

    def getComponentCost(self, component): #123 muss 3zahlen returnen
        chosen_value = self.instance.get_distance()[self.current_vertex,component]
        if (self.instance.get_goal_value()- (self.value+chosen_value))*(sum(self.chosen_coins)) + 1 <= 0:
            return 100000
        return (self.instance.get_goal_value()- (self.value+chosen_value))*(sum(self.chosen_coins)) + 1
        #return self.instance.getWeight(*[component])

    def get_chosen_coins(self):
        return self.chosen_coins

    def constructSolution(self):
        current_vertex = self.instance.getStartPoint()
        self.current_vertex = current_vertex

        while self.value < self.instance.get_goal_value():
            if current_vertex in self.instance.getGraph()[list(self.instance.getGraph().keys())[-1]]:
                break

            vertices = list(self.instance.getPossbileNextEdgesByVertex(current_vertex))

            #das ruft im weiteren dann get component cost auf,
            chosen_vertex = self.makeDecision(vertices)
            self.chosen_coins[chosen_vertex % len(self.instance.getGraph()[0])-1] += 1

            self.value += self.instance.distance[current_vertex,chosen_vertex]

            current_vertex = chosen_vertex
            self.current_vertex = current_vertex


def create_distance(d,N):
    """
    Creates distance matrix

    :param d: coin denomiations
    :param N: amount to sum up to
    :return: distance matrix, distance = coin value
    """

    size = (N // d[0]) * len(d) + 2
    matrix = np.zeros((size, size))

    c = 1
    for row in range(1, size - len(d) - 1):
        matrix[row, 1 + len(d) * c:1 + len(d) * (c + 1)] = d
        if ((row) % len(d)) == 0:
            c += 1

    #matrix[:, -1] = 1
    matrix[0, 1:1 + len(d)] = d
    matrix[-1, -1] = 0
    return matrix

def create_graph(d,N):
    graph = {
        0: np.arange(1,len(d)+1)
    }
    size = (N // d[0])*len(d) + 2
    c = 1
    for i in range(1, size - len(d) - 1):
        graph[i] = np.arange(len(d)*c + 1, len(d)*(c+1) + 1)
        if i % len(d) == 0:
            c += 1
    return graph

Dynamic Programming for benchmarking

In [2]:
def __coin_matrix__(coinList, changeNum):
    matrix=[[0 for m in range(changeNum+1)]for m in range(len(coinList)+1)]
    for i in range(changeNum+1):
        matrix[0][i]=i
    return matrix

def makeChange(coins,change):
    matrix=__coin_matrix__(coins,change)
    for c in range(1,len(coins)+1):
        for r in range(1,change+1):
            if coins[c-1]==r:
                matrix[c][r]=1
            elif coins[c-1] > r:
                matrix[c][r]=matrix[c-1][r]
            else:
                matrix[c][r] = min(matrix[c-1][r],1+matrix[c][r-coins[c-1]])

    return matrix[-1][-1] #<-- optimal number of coins


In [3]:
d=np.array([3,5,7])
N = 20
iterations = 20
times = np.empty(iterations)
dif = np.empty(iterations)
for i in range(iterations):
    start = time.time()
    distance = create_distance(d,N)
    graph = create_graph(d,N)
    instance = CPPInstance(50,distance,goal_value=N,graph=graph)
 

    obj, components = Solve(antCls=CPPAnt, instance=instance, numIterations=100, numAnts=10, alpha=1, beta=1)
    end = time.time()
    coin_values = []
    coins = np.zeros(len(d))
    for component in components:
        value = distance[:,component][distance[:,component] !=0][0]
        coin_values.append(value)
        coins[d == value] += 1

    print("Value: {} was created using {} coins ".format(sum(coin_values),len(coin_values)))
    print("Chosen coins: {}".format(coins))
    print("Runtime {}".format(end-start))
    times[i] = end-start
    dif[i]=len(coin_values)/makeChange(d,N)

4.0 4.0 4.0
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 4.0
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 500000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 4.0
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 

4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 4.0
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
Value: 20.0 was created using 4 coins 
Chosen coins: [2. 0. 2.]
Runtime 0.332655668258667
4.0 4.0 4.0
4

4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
Value: 20.0 was created using 4 coins 
Chosen coins: [2. 0. 2.]
Runtime 0.3221752643585205
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 600000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 4

4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 4.0
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 600000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 400000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 500000
4.0 4.0 4.0
4

KeyboardInterrupt: 

In [ ]:
print("Average runtime: {} \nDifference to optimum: {}".format(times.mean(),dif.mean()))

In [12]:
d=np.array([3,5,7,11,13,22])
N = 200

iterations = 10
times = np.empty(iterations)
dif = np.empty(iterations)
for i in range(iterations):
    start = time.time()
    distance = create_distance(d,N)
    graph = create_graph(d,N)
    instance = CPPInstance(50,distance,goal_value=N,graph=graph)
 

    obj, components = Solve(antCls=CPPAnt, instance=instance, numIterations=100, numAnts=10, alpha=1, beta=1)
    end = time.time()
    coin_values = []
    coins = np.zeros(len(d))
    for component in components:
        value = distance[:,component][distance[:,component] !=0][0]
        coin_values.append(value)
        coins[d == value] += 1

    print("Value: {} was created using {} coins ".format(sum(coin_values),len(coin_values)))
    print("Chosen coins: {}".format(coins))
    print("Runtime {}".format(end-start))
    times[i] = end-start
    dif[i]=len(coin_values)/makeChange(d,N)

13.0 13.0 1500000
13.0 13.0 1800000
14.0 13.0 1800000
13.0 13.0 1600000
14.0 13.0 1700000
12.0 12.0 1500000
11.0 11.0 1600000
12.0 11.0 1700000
12.0 11.0 2000000
13.0 11.0 1800000
11.0 11.0 1600000
15.0 11.0 1800000
12.0 11.0 1600000
12.0 11.0 1700000
14.0 11.0 1700000
13.0 11.0 1700000
12.0 11.0 1700000
14.0 11.0 1600000
12.0 11.0 1600000
11.0 11.0 1500000
12.0 11.0 1700000
12.0 11.0 1500000
13.0 11.0 1500000
12.0 11.0 1500000
11.0 11.0 1700000
12.0 11.0 1600000
12.0 11.0 1600000
12.0 11.0 1900000
10.0 10.0 1600000
12.0 10.0 1800000
13.0 10.0 1300000
10.0 10.0 1400000
10.0 10.0 1700000
16.0 10.0 1800000
12.0 10.0 1400000
12.0 10.0 1800000
12.0 10.0 1700000
12.0 10.0 1500000
14.0 10.0 1500000
12.0 10.0 1400000
13.0 10.0 1700000
12.0 10.0 1800000
13.0 10.0 1500000
14.0 10.0 1800000
12.0 10.0 1700000
12.0 10.0 1700000
14.0 10.0 1600000
10.0 10.0 1400000
12.0 10.0 1900000
14.0 10.0 1700000
12.0 10.0 1800000
10.0 10.0 1900000
13.0 10.0 1500000
13.0 10.0 1500000
13.0 10.0 1800000
15.0 10.0 

14.0 10.0 1900000
13.0 10.0 1600000
11.0 10.0 1600000
14.0 10.0 1700000
13.0 10.0 1800000
13.0 10.0 1800000
12.0 10.0 1800000
13.0 10.0 1800000
12.0 10.0 1800000
13.0 10.0 1600000
11.0 10.0 1500000
13.0 10.0 1600000
13.0 10.0 1800000
14.0 10.0 1800000
12.0 10.0 1600000
10.0 10.0 1500000
15.0 10.0 1700000
13.0 10.0 1500000
12.0 10.0 1600000
13.0 10.0 1500000
12.0 10.0 1600000
13.0 10.0 1700000
11.0 10.0 1700000
11.0 10.0 1600000
11.0 10.0 1700000
15.0 10.0 1600000
13.0 10.0 1800000
12.0 10.0 1700000
12.0 10.0 1600000
10.0 10.0 1900000
14.0 10.0 1700000
10.0 10.0 1500000
14.0 10.0 1800000
12.0 10.0 1500000
10.0 10.0 1700000
14.0 10.0 1700000
13.0 10.0 1600000
13.0 10.0 1800000
13.0 10.0 1700000
10.0 10.0 1700000
13.0 10.0 1800000
13.0 10.0 1600000
12.0 10.0 1700000
14.0 10.0 1700000
13.0 10.0 1700000
12.0 10.0 1600000
12.0 10.0 1500000
13.0 10.0 1600000
13.0 10.0 1700000
11.0 10.0 1700000
14.0 10.0 1500000
13.0 10.0 1500000
12.0 10.0 1500000
11.0 10.0 1600000
13.0 10.0 1600000
11.0 10.0 

13.0 10.0 1700000
12.0 10.0 1500000
11.0 10.0 1500000
11.0 10.0 1500000
13.0 10.0 1700000
11.0 10.0 1700000
11.0 10.0 1400000
12.0 10.0 1600000
13.0 10.0 1800000
12.0 10.0 1600000
12.0 10.0 1500000
13.0 10.0 1600000
14.0 10.0 1700000
12.0 10.0 1500000
15.0 10.0 1800000
12.0 10.0 1500000
12.0 10.0 1900000
11.0 10.0 2000000
13.0 10.0 1500000
11.0 10.0 1700000
13.0 10.0 1900000
13.0 10.0 1700000
15.0 10.0 1800000
12.0 10.0 1700000
Value: 200.0 was created using 10 coins 
Chosen coins: [0. 0. 0. 1. 1. 8.]
Runtime 2.7859721183776855
12.0 12.0 1500000
12.0 12.0 1600000
14.0 12.0 1800000
13.0 12.0 1600000
13.0 12.0 1700000
11.0 11.0 2000000
14.0 11.0 1500000
13.0 11.0 1700000
12.0 11.0 1600000
12.0 11.0 1700000
13.0 11.0 1700000
13.0 11.0 1400000
12.0 11.0 1700000
10.0 10.0 1500000
12.0 10.0 1700000
12.0 10.0 1700000
13.0 10.0 1300000
13.0 10.0 1600000
14.0 10.0 1800000
13.0 10.0 1600000
14.0 10.0 1800000
13.0 10.0 1800000
11.0 10.0 1400000
13.0 10.0 1500000
14.0 10.0 1900000
14.0 10.0 180000

In [13]:
print("Average runtime: {} \nDifference to optimum: {}".format(times.mean(),dif.mean()))

Average runtime: 2.7798283100128174 
Difference to optimum: 1.0


In [16]:
d=np.array([3,7,11,13,22,41,89,100,280])
#d=np.array([3,7,11,13,22,41,89,100,280,345,499,583,701,820,1023,1380])
N = 2000

iterations = 4
times = np.empty(iterations)
dif = np.empty(iterations)
for i in range(iterations):
    start = time.time()
    distance = create_distance(d,N)
    graph = create_graph(d,N)
    instance = CPPInstance(50,distance,goal_value=N,graph=graph)
 

    obj, components = Solve(antCls=CPPAnt, instance=instance, numIterations=100, numAnts=20, alpha=1, beta=1)
    end = time.time()
    coin_values = []
    coins = np.zeros(len(d))
    for component in components:
        value = distance[:,component][distance[:,component] !=0][0]
        coin_values.append(value)
        coins[d == value] += 1

    print("Value: {} was created using {} coins ".format(sum(coin_values),len(coin_values)))
    print("Chosen coins: {}".format(coins))
    print("Runtime {}".format(end-start))
    times[i] = end-start
    dif[i]=len(coin_values)/makeChange(d,N)

13.0 13.0 2600000
13.0 13.0 1900000
11.0 11.0 2300000
13.0 11.0 1900000
11.0 11.0 2200000
12.0 11.0 2300000
13.0 11.0 2200000
14.0 11.0 1800000
10.0 10.0 2000000
11.0 10.0 2000000
14.0 10.0 2000000
12.0 10.0 1800000
13.0 10.0 1900000
10.0 10.0 2300000
10.0 10.0 1900000
13.0 10.0 2000000
10.0 10.0 2000000
12.0 10.0 1700000
12.0 10.0 2100000
11.0 10.0 2200000
11.0 10.0 1900000
13.0 10.0 2100000
11.0 10.0 1800000
11.0 10.0 2100000
11.0 10.0 1700000
10.0 10.0 2000000
12.0 10.0 1900000
10.0 10.0 2200000
10.0 10.0 2000000
13.0 10.0 1800000
10.0 10.0 2000000
14.0 10.0 1700000
13.0 10.0 2000000
13.0 10.0 2100000
10.0 10.0 1800000
11.0 10.0 2100000
11.0 10.0 2100000
11.0 10.0 1900000
10.0 10.0 1700000
10.0 10.0 2400000
10.0 10.0 2500000
13.0 10.0 2000000
14.0 10.0 2100000
14.0 10.0 2000000
10.0 10.0 2100000
11.0 10.0 1700000
13.0 10.0 2100000
10.0 10.0 2100000
11.0 10.0 1900000
10.0 10.0 2500000
11.0 10.0 2200000
13.0 10.0 1800000
13.0 10.0 1700000
14.0 10.0 2000000
10.0 10.0 2000000
11.0 10.0 

In [15]:
print("Average runtime: {} \nDifference to optimum: {}".format(times.mean(),dif.mean()))

Average runtime: 11.808968365192413 
Difference to optimum: 1.0
